In [91]:
import random
from pulp import *
from collections import deque
import time
from collections import Counter
import networkx as nx

# Ucitavanje grafova

In [92]:
def load_graph_from_mtx(filename):
    graph = {}
    with open(filename, 'r') as file:
        for i, line in enumerate(file):
            # preskocimo prva tri reda (jer su tu podaci o broju grana, cvorova...)
            if i < 3:
                continue
            
            nodes = line.strip().split()
            node1, node2 = int(nodes[0]), int(nodes[1])

            if node1 not in graph:
                graph[node1] = set()
            if node2 not in graph:
                graph[node2] = set()

            graph[node1].add(node2)
            graph[node2].add(node1)

    return graph

def loadGraph(input_file):
    global d
    G = nx.Graph()
    for j in range(0,n):
        G.add_node(j)

    f = open(input_file, "r")
    string = f.readline()
    string = f.readline()
    string = f.readline()
    for i in range(0, m):
        string = f.readline()
        string = string.split()
        j = int(string[0])-1
        k = int(string[1])-1
        G.add_edge(j, k)
    f.close()
    d = []
    j = 1
    for i in range(n):
        d.append([float("inf")]*(j))
        j+=1
    for i in range(G.number_of_nodes()):
        ds = nx.single_source_shortest_path_length(G, i)
        for key in ds:
            if i >= key:
                d[i][key] = ds[key]
            else:
                d[key][i] = ds[key]
    print("n: " + str(G.number_of_nodes()))
    print("m: " + str(G.number_of_edges()))
    print("ncc: " + str(nx.number_connected_components(G)))


# Generisanje nasumicnih burning sekvenci

In [93]:

def generate_random_burning_sequence(graph):
    
    S=[] # sekvenca zapaljenih cvoreva
    B= set() # skup zapaljenih cvoreva
    NB= set(graph.keys()) # skup nezapaljenih cvoreva, inicajlno su to svi cvorevi grafa

    current_node = random.choice(list(NB))
    
    S.append(current_node)
    B.add(current_node)
    NB.remove(current_node)
    
    while len(B)<len(graph):
       
        # korak 2: zapaliti sve susjede (koji nisu zapaljeni) od cvorova u skupu zapaljenih, dodajemo ih u B, brisemo iz NB
        
        # Provjera: ako su svi cvorovi u NB zapravo cvorovi koji ce biti zapaljeni u narednom koraku (jer su direktni susjedi
        # zapaljenom cvoru nekom), onda cemo izabrati neki random cvor iz NB cisto kao simbol jos jednog koraka
        
        FB = []
        for b in B:
            neighbors_b1 = graph[b]
            for neighbor in neighbors_b1:
                if neighbor in NB:
                    if neighbor not in FB:
                        FB.append(neighbor)
             
        if set(FB) == NB:
            #print("fb",FB)
            #print("nb", NB)
            random_node = random.choice(list(NB))
          
            S.append(random_node)
          
        '''newlyburned = []
        for node in B:
            neighbors = graph[node]  # svi susjedi trenutnog zapaljenog cvora
            for neighbor in neighbors:
                if neighbor in NB:  # ako susjed nije zapaljen
                    newlyburned.append(neighbor)  # dodajemo ga u zapaljene
                    NB.remove(neighbor)  '''
        
        for new in FB:
            B.add(new)
            NB.remove(new)#dodajemo sve susjede u zapaljene
        
        # pravimo listu onih cvorova za koje znamo da ce se u iducem koraku zapaliti. Bolje da izaberemo sljedeci direktni
        # cvor za koji znamo da nece biti zapaljen u narednom koraku
        
        FB2 = []
        for b in B:
            neighbors_b = graph[b]
            for neighbor in neighbors_b:
                if neighbor in NB:
                    if neighbor not in FB2:
                        FB2.append(neighbor)
       
        # korak 3: pronaci iduci cvor koji ce biti direktno zapaljen, dodati ga u S, B i izbrisati iz NB
        
        max_burned = -1
        best_node = None
        candidates_for_best = []
        
        if NB:
            for node in NB:
                if node not in FB2: 
                    candidates_for_best.append(node)
            if candidates_for_best:
                best_node = random.choice(candidates_for_best)
            else:
                best_node = None        
                
            if best_node is not None:
                S.append(best_node)
                B.add(best_node)
                NB.remove(best_node)
            else:
             # Ako nema nijednog najboljeg cvora znaci da su svi preostali cvorovi povezani i mogu biti zapaljeni
             # Dodajemo slucajni cvor iz preostalih nezapaljenih čvorova
                random_node = random.choice(list(NB))
                S.append(random_node)
                B.add(random_node)
                NB.remove(random_node)
           
            
        else:
            break
           
    return S
        
        

def generate_multiple_random_burning_sequences(graph, num_sequences):
    
    sequences = []
    for _ in range(num_sequences):
        sequence = generate_random_burning_sequence(graph)
        if sequence is not None and sequence not in sequences:  
            sequences.append(sequence)
    return sequences


filename = "grafovi/manji/chesapeake.mtx"
graph = load_graph_from_mtx(filename)


num_sequences = 50

burning_sequences = generate_multiple_random_burning_sequences(graph, num_sequences)

# Ispis rezultata
print(f"Generisane burning sekvence:")
for seq in burning_sequences:
    print(seq,"len:", len(seq))



Generisane burning sekvence:
[5, 38, 2] len: 3
[2, 34, 15] len: 3
[12, 29, 27] len: 3
[34, 9, 26, 33] len: 4
[12, 9, 30] len: 3
[10, 34, 1] len: 3
[15, 37, 20] len: 3
[20, 37, 13] len: 3
[19, 34, 10] len: 3
[1, 25, 9] len: 3
[36, 34, 1] len: 3
[38, 4, 6, 3] len: 4
[3, 34, 38] len: 3
[26, 37, 32] len: 3
[8, 11, 19] len: 3
[33, 37, 25] len: 3
[28, 37, 30] len: 3
[3, 37, 38] len: 3
[35, 14, 4] len: 3
[36, 34, 8] len: 3
[6, 38, 34] len: 3
[32, 34, 37] len: 3
[8, 3, 36] len: 3
[1, 36, 28] len: 3
[5, 38, 4] len: 3
[33, 34, 26] len: 3
[2, 34, 18] len: 3
[34, 24, 17, 21] len: 4
[23, 16, 17] len: 3
[27, 34, 20] len: 3
[20, 37, 30] len: 3
[13, 5, 29] len: 3
[32, 37, 10] len: 3
[10, 34, 25] len: 3
[4, 38, 20] len: 3
[22, 3, 19] len: 3
[6, 38, 30] len: 3
[22, 10, 26] len: 3
[4, 38, 18] len: 3
[26, 37, 25] len: 3
[25, 34, 4] len: 3
[37, 21, 28, 3] len: 4
[10, 34, 29] len: 3
[7, 4, 3] len: 3
[27, 34, 7] len: 3
[32, 37, 25] len: 3
[5, 38, 20] len: 3
[13, 17, 29] len: 3
[15, 34, 11] len: 3


In [94]:
def transpose_list_of_lists(input_list):
    """
    Transponuje listu listi tako da prvi elementi svih podlisti budu u prvoj listi, drugi u drugoj, itd.
    """
    transposed = []
    for item in zip(*input_list):
        unique_item_list = list(set(item))
        transposed.append(unique_item_list)
    return transposed

burning_sequences_t = transpose_list_of_lists(burning_sequences)
print(burning_sequences_t)


[[1, 2, 3, 4, 5, 6, 7, 8, 10, 12, 13, 15, 19, 20, 22, 23, 25, 26, 27, 28, 32, 33, 34, 35, 36, 37, 38], [34, 3, 4, 37, 38, 36, 5, 9, 10, 11, 14, 16, 17, 21, 24, 25, 29], [1, 2, 3, 4, 6, 7, 8, 9, 10, 11, 13, 15, 17, 18, 19, 20, 25, 26, 27, 28, 29, 30, 32, 34, 36, 37, 38]]


# PuLP

In [95]:
    
def ILP(L,k):
    global d  
    
    #model
    model = LpProblem("ILP_COV", LpMinimize)
    
    #definisanje varijabli
    x = LpVariable.dicts('x', (range(0, k), range(0, n+1)), cat='Binary')
    b = LpVariable.dict('b',range(0, n+1), cat='Binary')
    
    #funkcija cilja
    model += lpSum(x[i][j] for i in range(0, k) for j in range(0, n+1))
    
    #domensko ogranicenje
    for i in range(int(k/2)+1):
        print(burning_sequences_t[i])
        for j in burning_sequences_t[i]:
            model += lpSum(x[i][j]) == 1 
    
    #ogranicenja
    #(13)
    for i in range(k):
        suma=0
        for j in range(n+1):
            suma+=x[i][j]
        model +=(suma<=1)
        
    #(14)
    for j in range(n):
        suma = 0
        for i in range(k):
            for kk in range(n):
                if kk >= j:
                    if d[kk][j] <= i:
                        suma += x[i][kk]
                else:
                    if d[j][kk] <= i:
                        suma += x[i][kk]
        model +=(b[j]<=suma)
        
    #(12)
    for i in range(k):
        sum2 = 0
        if i == 0:
            sum1 = 1
        else:
            sum1 = 0
        for j in range(n+1):
            sum2 += x[i][j]
            if i >= 1:
                sum1 += x[i-1][j]
        model +=(sum2<=sum1)

    
    #(15)
    model +=lpSum(b[j] for j in range(0, n+1)) == n 
    
    
    # rjesavanje
    model.solve(PULP_CBC_CMD(maxSeconds=600, msg=True, fracGap=0))
    

    if LpStatus[model.status] == 'Optimal':
        print(f"Optimal Solution Found: {model.objective.value()}")
        for i in range(k):
            for j in range(n+1):
                if x[i][j].value() == 1:
                    print(f"x[{i}, {j}] = 1")
    else:
        print("No optimal solution found")
    s = []
    
    x_out = [[0]*n for i in range(k)]
    for r in range(k):
        i = 0
        for e in x_out[r]:
            if e == 1:
                s.append(i)
                break
            i += 1
 
    s.reverse()
   

    return s
        
def main(ni, mi, input_file, Li, Ui):
    n = ni
    m = mi
    U = Ui
    L = Li
    loadGraph(input_file)
    s = ILP(L,U)

if __name__ == "__main__":
    bs = []
    n = 0
    m = 0
    d = {}
    bs_size = float("inf")
    folder_dataset = 'grafovi/manji/'
    dataset = [
        #['karate.mtx',34,78,2,3] #['dolphins.mtx',62,159,2,4],
        ['chesapeake.mtx',39,170,1,3], # instance, n, m, l, h
        #['dolphins.mtx',62,159,2,4],
        #['rt-retweet.mtx',96,117,2,5],
        #['polbooks.mtx',105,441,2,4],
        #['adjnoun.mtx',112,425,2,4],
        #['ia-infect-hyper.mtx',113,2196,1,3],
        #['C125-9.mtx',125,6963,1,3],
        #['ia-enron-only.mtx',143,623,2,4],
        #['c-fat200-1.mtx',200,1534,3,7],
        #['c-fat200-2.mtx',200,3235,2,5],
        #['c-fat200-5.mtx',200,8473,1,3],
        ]
    for i in range(len(dataset)):
        print("________________________________________________")
        instance = dataset[i][0]
        print("instance: " + instance)
        n = dataset[i][1]
        m = dataset[i][2]
        L = dataset[i][3]
        U = dataset[i][4]
        main(n, m, folder_dataset + dataset[i][0], L, U)
    
    

________________________________________________
instance: chesapeake.mtx
n: 39
m: 170
ncc: 1
[1, 2, 3, 4, 5, 6, 7, 8, 10, 12, 13, 15, 19, 20, 22, 23, 25, 26, 27, 28, 32, 33, 34, 35, 36, 37, 38]
[34, 3, 4, 37, 38, 36, 5, 9, 10, 11, 14, 16, 17, 21, 24, 25, 29]
No optimal solution found
